# Correcting for multiple comparisons

## Get proportion of significant tracts after correcting for multiple comparisons:
Using FDR.


In [1]:
import pandas as pd
import numpy as np
from statsmodels.stats.multitest import multipletests as correct_multi

In [2]:
indir = "/cbica/projects/csdsi/cleaned_paper_analysis/bug_fix/data/dice_scores/permutation_stats/"
cs_acqs = ["HA-SC92+55-1", "HA-SC92+55-2",  "HA-SC92", "HA-SC55-1",  "HA-SC55-2", "RAND57"]
mtrk = "all_tracks"
trks = ["Arcuate_Fasciculus_L", "Arcuate_Fasciculus_R", "Cingulum_Frontal_Parahippocampal_L", "Cingulum_Frontal_Parahippocampal_R", "Cingulum_Frontal_Parietal_L", "Cingulum_Frontal_Parietal_R", "Cingulum_Parahippocampal_L", "Cingulum_Parahippocampal_Parietal_L", "Cingulum_Parahippocampal_Parietal_R", "Cingulum_Parahippocampal_R", "Cingulum_Parolfactory_L", "Cingulum_Parolfactory_R", "Corpus_Callosum_Body", "Corpus_Callosum_Forceps_Major", "Corpus_Callosum_Forceps_Minor", "Corpus_Callosum_Tapetum", "Corticospinal_Tract_L", "Corticospinal_Tract_R", "Corticostriatal_Tract_Anterior_L", "Corticostriatal_Tract_Anterior_R", "Corticostriatal_Tract_Posterior_L", "Corticostriatal_Tract_Posterior_R", "Corticostriatal_Tract_Superior_L", "Corticostriatal_Tract_Superior_R", "Fornix_L", "Fornix_R", "Frontal_Aslant_Tract_L", "Frontal_Aslant_Tract_R", "Inferior_Fronto_Occipital_Fasciculus_L", "Inferior_Fronto_Occipital_Fasciculus_R", "Inferior_Longitudinal_Fasciculus_L", "Inferior_Longitudinal_Fasciculus_R", "Middle_Longitudinal_Fasciculus_L", "Middle_Longitudinal_Fasciculus_R", "Optic_Radiation_L", "Optic_Radiation_R", "Parietal_Aslant_Tract_L", "Parietal_Aslant_Tract_R", "Reticular_Tract_L", "Reticular_Tract_R", "Superior_Longitudinal_Fasciculus1_L", "Superior_Longitudinal_Fasciculus1_R", "Superior_Longitudinal_Fasciculus2_L", "Superior_Longitudinal_Fasciculus2_R", "Superior_Longitudinal_Fasciculus3_L", "Superior_Longitudinal_Fasciculus3_R", "Thalamic_Radiation_Anterior_L", "Thalamic_Radiation_Anterior_R", "Thalamic_Radiation_Posterior_L", "Thalamic_Radiation_Posterior_R", "Thalamic_Radiation_Superior_L", "Thalamic_Radiation_Superior_R", "Uncinate_Fasciculus_L", "Uncinate_Fasciculus_R", "Vertical_Occipital_Fasciculus_L", "Vertical_Occipital_Fasciculus_R"]

def correct_multiple_comparisons(grp, method="fdr_bh"):
    stats_df = pd.read_csv(indir+grp+"/"+mtrk+"/subject_medians_all_stats.csv")
    alltrk_p_df = pd.DataFrame(index=cs_acqs)

    for acq in cs_acqs:
        acq_med = stats_df[stats_df["Acquisition"]==acq]["Subject Median"].median()
        alltrk_p_df.loc[acq, "Median Median Difference"] = round(acq_med, 3)

        # Get p-value props:
        p_values_raw = np.array(stats_df[stats_df["Acquisition"]==acq]["p-value"].astype(float))
        _, p_values, _, _ = correct_multi(np.array(p_values_raw), method=method)

        alltrk_p_df.loc[acq, "n(p < 0.05) [corrected]"] = np.count_nonzero([p_values < 0.05]) 
    alltrk_p_df.to_csv(indir+grp+"/"+mtrk+"/tracks_summary_proportionandmax_multcorrected_"+method+"_v2.csv")
    return alltrk_p_df

def get_raw_p_values(grp):
    p_df = pd.read_csv(indir+grp+"/"+mtrk+"/tracks_summary_proportionandmax_v2.csv")
    return p_df

### Retrospective within accuracy
Supplementary Table 2

In [3]:
correct_multiple_comparisons("retro_wthn_acc") 

,Median Median Difference,n(p < 0.05) [corrected]
HA-SC92+55-1,-0.013,6.0
HA-SC92+55-2,-0.017,6.0
HA-SC92,0.002,0.0
HA-SC55-1,0.039,54.0
HA-SC55-2,0.058,56.0
RAND57,0.074,56.0


### Retrospective between accuracy
Supplementary Table 4

In [4]:
correct_multiple_comparisons("retro_btwn_acc_unpaired")

,Median Median Difference,n(p < 0.05) [corrected]
HA-SC92+55-1,0.024,56.0
HA-SC92+55-2,0.022,56.0
HA-SC92,0.034,56.0
HA-SC55-1,0.060,56.0
HA-SC55-2,0.074,56.0
RAND57,0.089,56.0


### Retrospective between reliability
Supplementary Table 5

In [8]:
correct_multiple_comparisons("retro_btwn_rel")

,Median Median Difference,n(p < 0.05) [corrected]
HA-SC92+55-1,0.008,0.0
HA-SC92+55-2,0.009,0.0
HA-SC92,0.018,0.0
HA-SC55-1,0.038,0.0
HA-SC55-2,0.039,0.0
RAND57,0.063,0.0


## Same thing for scalars:

In [2]:
indir = "/cbica/projects/csdsi/cleaned_paper_analysis/bug_fix/data/pearson_correlations/permutation_stats/"
cs_acqs = ["HA-SC92+55-1", "HA-SC92+55-2",  "HA-SC92", "HA-SC55-1",  "HA-SC55-2", "RAND57"]
metrics = ["nqa", "gfa", "iso"]
def make_all_scalar_df(grp):
    p_df = pd.DataFrame(columns=cs_acqs, index=metrics)
    med_df = pd.DataFrame(columns=cs_acqs, index=metrics)
    for met in metrics:
        stats_df = pd.read_csv(indir+grp+"/"+met+"_mask/subject_medians_all_stats.csv")
        stats_df = stats_df.set_index("Acquisition")
        for acq in cs_acqs:
            p_df.loc[met, acq] = stats_df.loc[acq, "p-value"]
            med_df.loc[met, acq] = stats_df.loc[acq, "Subject Median"]
    return p_df, med_df


def correct_multiple_comparisons(grp, method="fdr_bh"):
    p_df, med_df = make_all_scalar_df(grp)
    p_df_corr = pd.DataFrame(columns=cs_acqs, index=metrics)
    for acq in cs_acqs:
        # Get p-value props:
        p_values_raw = p_df[acq]
        _, p_values, _, _ = correct_multi(np.array(p_values_raw), method=method)
        p_df_corr[acq] = p_values
    
    for met in metrics:
        stats_df = pd.DataFrame(index=cs_acqs)
        for acq in cs_acqs:
            stats_df.loc[acq, "Median Difference"] = round(med_df.loc[met, acq], 3)
            stats_df.loc[acq, "p-value"] = round(p_df_corr.loc[met, acq], 3)
        stats_df.to_csv(indir+grp+"/"+met+"_mask/statssummary_multcorrected_"+method+".csv")
        print(met)
        print(stats_df)
        print("\n")

In [3]:
correct_multiple_comparisons("retro_wthn_acc") #Supplementary Table 6

nqa
              Median Difference  p-value
HA-SC92+55-1             -0.018    0.008
HA-SC92+55-2             -0.018    0.008
HA-SC92                  -0.015    0.009
HA-SC55-1                 0.003    0.119
HA-SC55-2                 0.022    0.015
RAND57                    0.048    0.015


gfa
              Median Difference  p-value
HA-SC92+55-1              0.009    0.188
HA-SC92+55-2              0.008    0.200
HA-SC92                   0.014    0.120
HA-SC55-1                 0.046    0.117
HA-SC55-2                 0.065    0.038
RAND57                    0.144    0.015


iso
              Median Difference  p-value
HA-SC92+55-1              0.010    0.116
HA-SC92+55-2              0.012    0.088
HA-SC92                   0.011    0.093
HA-SC55-1                 0.010    0.119
HA-SC55-2                 0.015    0.038
RAND57                    0.005    0.162




In [4]:
correct_multiple_comparisons("retro_btwn_acc_unpaired") #Supplementary Table 7

nqa
              Median Difference  p-value
HA-SC92+55-1              0.009    0.000
HA-SC92+55-2              0.008    0.001
HA-SC92                   0.012    0.000
HA-SC55-1                 0.029    0.000
HA-SC55-2                 0.047    0.001
RAND57                    0.075    0.001


gfa
              Median Difference  p-value
HA-SC92+55-1              0.034    0.020
HA-SC92+55-2              0.034    0.020
HA-SC92                   0.039    0.012
HA-SC55-1                 0.068    0.002
HA-SC55-2                 0.090    0.001
RAND57                    0.170    0.001


iso
              Median Difference  p-value
HA-SC92+55-1              0.035      0.0
HA-SC92+55-2              0.036      0.0
HA-SC92                   0.036      0.0
HA-SC55-1                 0.034      0.0
HA-SC55-2                 0.039      0.0
RAND57                    0.031      0.0




In [22]:
correct_multiple_comparisons("retro_btwn_rel") #Supplementary Table 8

nqa
              Median Difference  p-value
HA-SC92+55-1              0.003    0.123
HA-SC92+55-2              0.003    0.127
HA-SC92                   0.007    0.131
HA-SC55-1                 0.021    0.128
HA-SC55-2                 0.028    0.126
RAND57                    0.049    0.135


gfa
              Median Difference  p-value
HA-SC92+55-1             -0.042    0.103
HA-SC92+55-2             -0.042    0.090
HA-SC92                  -0.038    0.104
HA-SC55-1                -0.019    0.114
HA-SC55-2                -0.009    0.126
RAND57                    0.026    0.135


iso
              Median Difference  p-value
HA-SC92+55-1              0.004    0.103
HA-SC92+55-2              0.004    0.090
HA-SC92                   0.004    0.104
HA-SC55-1                 0.005    0.114
HA-SC55-2                 0.006    0.126
RAND57                    0.007    0.135




## Median full DSI reliability
For supplementary table 3

In [12]:
trks = ["Arcuate_Fasciculus_L", "Arcuate_Fasciculus_R", "Cingulum_Frontal_Parahippocampal_L", "Cingulum_Frontal_Parahippocampal_R", "Cingulum_Frontal_Parietal_L", "Cingulum_Frontal_Parietal_R", "Cingulum_Parahippocampal_L", "Cingulum_Parahippocampal_Parietal_L", "Cingulum_Parahippocampal_Parietal_R", "Cingulum_Parahippocampal_R", "Cingulum_Parolfactory_L", "Cingulum_Parolfactory_R", "Corpus_Callosum_Body", "Corpus_Callosum_Forceps_Major", "Corpus_Callosum_Forceps_Minor", "Corpus_Callosum_Tapetum", "Corticospinal_Tract_L", "Corticospinal_Tract_R", "Corticostriatal_Tract_Anterior_L", "Corticostriatal_Tract_Anterior_R", "Corticostriatal_Tract_Posterior_L", "Corticostriatal_Tract_Posterior_R", "Corticostriatal_Tract_Superior_L", "Corticostriatal_Tract_Superior_R", "Fornix_L", "Fornix_R", "Frontal_Aslant_Tract_L", "Frontal_Aslant_Tract_R", "Inferior_Fronto_Occipital_Fasciculus_L", "Inferior_Fronto_Occipital_Fasciculus_R", "Inferior_Longitudinal_Fasciculus_L", "Inferior_Longitudinal_Fasciculus_R", "Middle_Longitudinal_Fasciculus_L", "Middle_Longitudinal_Fasciculus_R", "Optic_Radiation_L", "Optic_Radiation_R", "Parietal_Aslant_Tract_L", "Parietal_Aslant_Tract_R", "Reticular_Tract_L", "Reticular_Tract_R", "Superior_Longitudinal_Fasciculus1_L", "Superior_Longitudinal_Fasciculus1_R", "Superior_Longitudinal_Fasciculus2_L", "Superior_Longitudinal_Fasciculus2_R", "Superior_Longitudinal_Fasciculus3_L", "Superior_Longitudinal_Fasciculus3_R", "Thalamic_Radiation_Anterior_L", "Thalamic_Radiation_Anterior_R", "Thalamic_Radiation_Posterior_L", "Thalamic_Radiation_Posterior_R", "Thalamic_Radiation_Superior_L", "Thalamic_Radiation_Superior_R", "Uncinate_Fasciculus_L", "Uncinate_Fasciculus_R", "Vertical_Occipital_Fasciculus_L", "Vertical_Occipital_Fasciculus_R"]
median_df = pd.DataFrame(index = trks)
for trk in trks:
    df = pd.read_csv("/cbica/projects/csdsi/cleaned_paper_analysis/bug_fix/data/dice_scores/retro_fulldsi_btwn_rel/"+trk+"/"+"/all_subjects.csv")
    median_sc = np.median(np.array(df.drop(["Unnamed: 0", "Subject"], axis=1))[np.triu_indices(8, k = 1)])
    median_df.loc[trk, "Full DSI Reliability"] = median_sc

In [13]:
supp_df = median_df.sort_values("Full DSI Reliability", ascending=False).round(3)

In [14]:
supp_df

,Full DSI Reliability
Corpus_Callosum_Forceps_Minor,0.881
Corpus_Callosum_Body,0.867
Parietal_Aslant_Tract_R,0.858
Corpus_Callosum_Forceps_Major,0.852
Vertical_Occipital_Fasciculus_R,0.837
Corticostriatal_Tract_Superior_R,0.834
Corticostriatal_Tract_Anterior_L,0.834
Parietal_Aslant_Tract_L,0.833
Corticostriatal_Tract_Superior_L,0.833
Inferior_Longitudinal_Fasciculus_L,0.832


In [17]:
supp_df.to_csv("/cbica/projects/csdsi/cleaned_paper_analysis/bug_fix/data/dice_scores/retro_fulldsi_btwn_rel/supp3.csv")